In [1]:
pip install eventlet


  Using cached eventlet-0.33.0-py2.py3-none-any.whl (226 kB)
  Using cached dnspython-2.2.1-py3-none-any.whl (269 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install flask-socketio


     |████████████████████████████████| 56 kB 3.7 MB/s eta 0:00:011
     |████████████████████████████████| 52 kB 1.8 MB/s  eta 0:00:01
  Using cached bidict-0.22.0-py3-none-any.whl (36 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import eventlet
import socketio
import cv2
import io
import numpy as np
from PIL import Image
import base64

from keras.preprocessing.image import img_to_array
from keras.models import load_model
import numpy as np
import argparse
import imutils
from imutils.video import VideoStream
from imutils import face_utils
import cv2

from threading import Thread
import numpy as np
import time
import dlib

HAARCASCADE = "haarcascade_frontalface_default.xml"
MODEL = "model.h5"

# load the face detector cascade and smile detector CNN
detector = cv2.CascadeClassifier(HAARCASCADE)
dlib_detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
model = load_model(MODEL)

sio = socketio.Server(cors_allowed_origins='*')

@sio.event
def my_message(data):
    print('message received with ', data)
    sio.emit('my response', {'response': 'my response'})

@sio.on('image')
def on_message(arg1, data_image):
    start_time = time.time()
    sbuf = io.StringIO()
    sbuf.write(data_image)

    # decode and convert into image
    b = io.BytesIO(base64.b64decode(data_image))
    pimg = Image.open(b)

    ## converting RGB to BGR, as opencv standards
    frame = cv2.cvtColor(np.array(pimg), cv2.COLOR_RGB2BGR)
    # emit the frame back
    frame = imutils.resize(frame, width = 300)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # detect faces in the input frame, then clone the frame so
    # that we can draw on it
    rects = detector.detectMultiScale(gray, scaleFactor = 1.1, minNeighbors = 5,
        minSize = (30, 30), flags = cv2.CASCADE_SCALE_IMAGE)

    label_written = False

    for (fX, fY, fW, fH) in rects:
        # extract the ROI of the face from the grayscale image,
        # resize it to a fixed 28x28 pixels, and then prepare the
        # ROI for classification via CNN
        roi = gray[fY: fY + fH, fX: fX + fW]
        roi = cv2.resize(roi, (28, 28))
        roi = roi.astype("float") / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis = 0)

        # determine the probabilities of both "smiling" and "not similing"
        # then set the label accordingly
        (notSmiling, smiling) = model.predict(roi)[0]
        label = "E" if smiling > notSmiling else "Others"
        # print(label)

        if not label_written:
            cv2.putText(frame, label + " detected", (10, 30),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),2)
            label_written = True

    rects = dlib_detector(gray, 0)
    for rect in rects:
		# determine the facial landmarks for the face region, then convert the facial landmark (x, y)-coordinates to a NumPy array
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)

		# draw lip contours
        lip = shape[48:60]
        cv2.drawContours(frame, [lip], -1, (0, 255, 0), 1)

    # frame = cv2.flip(frame, 1)
    imgencode = cv2.imencode('.jpg', frame)[1]
    # cv2.imshow("Frame", frame)
    # cv2.waitKey(0)
    # print(arg1)

    # base64 encode
    stringData = base64.b64encode(imgencode).decode('utf-8')
    b64_src = 'data:image/jpg;base64,'
    stringData = b64_src + stringData

    sio.emit('response_back', stringData)
    end_time = time.time()
    

# wrap with a WSGI application
app = socketio.WSGIApp(sio)

if __name__ == '__main__':
    eventlet.wsgi.server(eventlet.listen(('localhost', 5000)), app)


2022-04-30 13:12:43.330564: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
(1658) wsgi starting up on http://127.0.0.1:5000
